In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
with open('../data/my_val.pkl3', 'rb') as f:
    data = joblib.load(f)
locals().update(data)
del data
np.random.seed(1234)

test = pd.read_csv('../data/test_ver04.csv', index_col='STU_ID')
test = test.drop(columns='F3ONET2CURR')

In [3]:
from sklearn.preprocessing import MinMaxScaler

In [4]:
minmax = MinMaxScaler()
x_tr = minmax.fit_transform(x)
val_x_tr = minmax.fit_transform(val_x)
test_tr = minmax.fit_transform(test)

In [5]:
y.unique()

array([33, 13, 35, 11, 53, 41, 29, 25, 31, 15, 43, 27, 49, 39, 51, 21, 47,
       17, 37, 19], dtype=int64)

In [6]:
idx2class = {}
for i, v in enumerate(y.unique()):
    idx2class[i] = v
idx2class

{0: 33,
 1: 13,
 2: 35,
 3: 11,
 4: 53,
 5: 41,
 6: 29,
 7: 25,
 8: 31,
 9: 15,
 10: 43,
 11: 27,
 12: 49,
 13: 39,
 14: 51,
 15: 21,
 16: 47,
 17: 17,
 18: 37,
 19: 19}

In [7]:
class2idx = {v:i for i,v in idx2class.items()}
class2idx

{33: 0,
 13: 1,
 35: 2,
 11: 3,
 53: 4,
 41: 5,
 29: 6,
 25: 7,
 31: 8,
 15: 9,
 43: 10,
 27: 11,
 49: 12,
 39: 13,
 51: 14,
 21: 15,
 47: 16,
 17: 17,
 37: 18,
 19: 19}

In [8]:
y_tr = y.replace(class2idx)
y_tr

STU_ID
385113     0
429104     1
190216     2
386221     3
245217     4
          ..
450116     9
419116     2
112110     7
211119    11
173201     3
Name: F3ONET2CURR, Length: 7466, dtype: int64

In [9]:
val_y_tr = val_y.replace(class2idx)
val_y_tr

STU_ID
282111     8
401108    14
289228     6
382211     9
107206     4
          ..
437110     7
276202     9
266208     7
349240    16
249219    17
Name: F3ONET2CURR, Length: 2489, dtype: int64

In [33]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense,BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [11]:
x_tr.shape

(7466, 151)

In [15]:
y_tr.nunique()

20

In [34]:
model = Sequential([
              Input(shape=(x_tr.shape[1],)),
              Dense(128, activation ='relu',kernel_regularizer=l2(0.001), kernel_initializer='he_normal',use_bias=False),
              Dropout(0.2),
              Dense(64, activation = 'relu', kernel_initializer='he_normal', use_bias=False),
              Dropout(0.2),
              Dense(32, activation ='relu', kernel_initializer='he_normal'),
              Dense(y_tr.nunique(), activation='softmax')])

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 128)               19328     
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 64)                8192      
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_24 (Dense)             (None, 20)                660       
Total params: 30,260
Trainable params: 30,260
Non-trainable params: 0
__________________________________________________

In [35]:
epochs = 20
batch = 32
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,
              metrics=['acc'])
model.fit(x_tr, y_tr, batch_size=batch, epochs=epochs)

Epoch 1/20
234/234 [==============================] - 0s 2ms/step - loss: 3.0089 - acc: 0.1268
Epoch 2/20
234/234 [==============================] - 0s 2ms/step - loss: 2.8147 - acc: 0.1606
Epoch 3/20
234/234 [==============================] - 0s 2ms/step - loss: 2.7058 - acc: 0.1844
Epoch 4/20
234/234 [==============================] - 0s 2ms/step - loss: 2.6255 - acc: 0.2020
Epoch 5/20
234/234 [==============================] - 0s 2ms/step - loss: 2.5853 - acc: 0.2085
Epoch 6/20
234/234 [==============================] - 0s 2ms/step - loss: 2.5526 - acc: 0.2156
Epoch 7/20
234/234 [==============================] - 0s 2ms/step - loss: 2.5342 - acc: 0.2147
Epoch 8/20
234/234 [==============================] - 0s 2ms/step - loss: 2.5116 - acc: 0.2235
Epoch 9/20
234/234 [==============================] - 0s 2ms/step - loss: 2.4858 - acc: 0.2340
Epoch 10/20
234/234 [==============================] - 0s 2ms/step - loss: 2.4729 - acc: 0.2351
Epoch 11/20
234/234 [============================

In [36]:
model.evaluate(val_x_tr, val_y_tr)

78/78 [==============================] - 0s 1ms/step - loss: 2.5361 - acc: 0.2314


[2.536062240600586, 0.23141823709011078]

In [37]:
pred = model.predict(test_tr)
pred

array([[0.02268364, 0.05022664, 0.10373957, ..., 0.00683108, 0.01887235,
        0.00604259],
       [0.06996372, 0.01667635, 0.06662744, ..., 0.01132424, 0.04096812,
        0.00258895],
       [0.04867441, 0.1146619 , 0.02490994, ..., 0.00733888, 0.00506355,
        0.03243866],
       ...,
       [0.02001639, 0.02757076, 0.10229938, ..., 0.00294775, 0.03046317,
        0.00506004],
       [0.00993502, 0.05182706, 0.03177714, ..., 0.00052144, 0.00555692,
        0.00230317],
       [0.05892349, 0.02887655, 0.03564925, ..., 0.10629475, 0.00946513,
        0.01712431]], dtype=float32)

In [38]:
pred_class = [idx2class[i] for i in pred.argmax(axis=1)]
pred_class

[43,
 49,
 43,
 43,
 51,
 13,
 11,
 17,
 49,
 43,
 43,
 27,
 13,
 43,
 13,
 13,
 43,
 43,
 51,
 43,
 25,
 13,
 25,
 21,
 13,
 31,
 43,
 13,
 43,
 43,
 49,
 43,
 35,
 27,
 43,
 29,
 29,
 43,
 43,
 17,
 43,
 11,
 51,
 43,
 29,
 43,
 29,
 29,
 47,
 25,
 43,
 43,
 43,
 49,
 13,
 43,
 43,
 43,
 43,
 43,
 43,
 29,
 43,
 31,
 41,
 43,
 53,
 43,
 47,
 13,
 43,
 43,
 43,
 43,
 49,
 43,
 43,
 43,
 43,
 17,
 43,
 27,
 11,
 43,
 43,
 43,
 49,
 41,
 47,
 11,
 49,
 15,
 43,
 49,
 29,
 43,
 13,
 29,
 47,
 29,
 43,
 13,
 29,
 29,
 43,
 29,
 13,
 47,
 43,
 13,
 43,
 51,
 43,
 17,
 29,
 51,
 17,
 13,
 43,
 29,
 27,
 25,
 29,
 43,
 13,
 43,
 17,
 43,
 29,
 21,
 13,
 29,
 13,
 43,
 43,
 33,
 43,
 17,
 25,
 11,
 47,
 51,
 43,
 11,
 43,
 47,
 43,
 29,
 43,
 29,
 43,
 43,
 27,
 43,
 29,
 43,
 43,
 43,
 43,
 13,
 43,
 29,
 13,
 43,
 11,
 49,
 11,
 35,
 25,
 25,
 29,
 53,
 15,
 25,
 43,
 47,
 13,
 11,
 29,
 43,
 29,
 25,
 25,
 11,
 43,
 43,
 43,
 19,
 43,
 43,
 29,
 29,
 43,
 43,
 43,
 51,
 43,
 19,
 33,
 13,


In [39]:
submission = pd.DataFrame(pred_class, index=test.index, columns=['PRED']).reset_index()
submission

,STU_ID,PRED
0,310221,43
1,228218,49
2,179231,43
3,215101,43
4,245121,51
...,...,...
2474,439214,25
2475,106206,43
2476,272131,43
2477,350228,43


In [40]:
submission.to_csv('my_pred38.csv', index=False)